In [1]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [3]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [4]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

(43266, 10)


In [5]:
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
df_test['tag'] = 'test'
print df_test.shape

(18543, 9)


In [6]:
df = pd.concat([df_train,df_test],axis= 0)
print df.shape
df.head()

(61809, 10)


/home/prashant/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,Company-response,Complaint-ID,Complaint-Status,Complaint-reason,Consumer-complaint-summary,Consumer-disputes,Date-received,Date-sent-to-company,Transaction-Type,tag
0,NaN,Tr-1,Closed with explanation,"Loan servicing, payments, escrow account","Seterus, Inc a déposé un faux rapport auprès d...",Yes,11/11/2015,11/11/2015,Mortgage,train
1,Company chooses not to provide a public response,Tr-2,Closed with non-monetary relief,Incorrect information on credit report,XX / XX / XXXX La requête en faillite n ° XXXX...,No,7/7/2015,7/7/2015,Credit reporting,train
2,NaN,Tr-3,Closed with explanation,Using a debit or ATM card,"El XXXX / XXXX / 15, estaba preparando el vuel...",No,5/7/2015,5/7/2015,Bank account or service,train
3,Company believes it acted appropriately as aut...,Tr-4,Closed with explanation,Cont'd attempts collect debt not owed,"The loan was paid in XXXX XXXX. In XXXX, 4 yea...",No,11/12/2016,11/12/2016,Debt collection,train
4,Company has responded to the consumer and the ...,Tr-5,Closed with explanation,Payoff process,J'ai obtenu un compte de crédit de soins pour ...,No,9/29/2016,9/29/2016,Credit card,train


In [7]:
df['Complaint-Status'].value_counts()

Closed with explanation            34300
Closed with non-monetary relief     5018
Closed with monetary relief         2818
Closed                               809
Untimely response                    321
Name: Complaint-Status, dtype: int64

In [8]:
df['Transaction-Type'] = df['Transaction-Type'].map(lambda x : x.lower().replace(' ','_'))

In [9]:
df['Complaint-reason'] = df['Complaint-reason'].apply(preprocess_data)

In [10]:
text = list(df['Complaint-reason'])
vectorizer = TfidfVectorizer(min_df=1)
# tokenize and build vocab
vectorizer.fit(text)
# encode document
vector = vectorizer.transform(text)
# summarize encoded vector
print(vector.shape)
#print(vector.toarray())

(61809, 198)


In [ ]:
# import time

# from sklearn.manifold import TSNE
# n_sne = 7000
# time_start = time.time()
# tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
# tsne_results = tsne.fit_transform(vector.toarray())


In [11]:
vec_df = pd.DataFrame(vector.toarray())

In [12]:
print vec_df.shape

(61809, 198)


In [13]:
vec_df['tag'] = list(df['tag'])
vec_df['Complaint-Status'] = list(df['Complaint-Status'])
vec_df['Transaction-Type'] = list(df['Transaction-Type'])
vec_df['Complaint-ID'] = list(df['Complaint-ID'])

In [14]:
vec_df['cd'] = list(df['Consumer-disputes'])

In [15]:
complete_df_one_hot = vec_df

In [16]:
train_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'train']
train_data.drop(['tag'],inplace=True,axis=1)
test_data = complete_df_one_hot.loc[complete_df_one_hot['tag']== 'test']
test_data.drop(['tag'],inplace=True,axis =1)


/home/prashant/.local/lib/python2.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [17]:
shuf_dta = train_data.sample(frac=1)
shuf_dta.shape

(43266, 202)

In [18]:
y_train = (shuf_dta['Complaint-Status'])
x_train = shuf_dta.drop(['Complaint-Status','Complaint-ID'],axis =1)

In [19]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,Transaction-Type,cd
29880,0.373422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,bank_account_or_service,No
23505,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,debt_collection,No
42701,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,credit_reporting,No
35679,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,mortgage,No
36005,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,student_loan,NaN


In [21]:
x_train = pd.get_dummies(x_train,prefix=['Transaction-Type','cd'])

x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,Transaction-Type_mortgage,Transaction-Type_other_financial_service,Transaction-Type_payday_loan,"Transaction-Type_payday_loan,_title_loan,_or_personal_loan",Transaction-Type_prepaid_card,Transaction-Type_student_loan,Transaction-Type_vehicle_loan_or_lease,Transaction-Type_virtual_currency,cd_No,cd_Yes
29880,0.373422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
23505,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
42701,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
35679,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,1,0
36005,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0


In [22]:
print y_train.shape
y_train_one_hot = pd.get_dummies(y_train,prefix=['Complaint-Status'])
print y_train_one_hot.shape

(43266,)
(43266, 5)


In [23]:
x_train.head()

,0,1,2,3,4,5,6,7,8,9,...,Transaction-Type_mortgage,Transaction-Type_other_financial_service,Transaction-Type_payday_loan,"Transaction-Type_payday_loan,_title_loan,_or_personal_loan",Transaction-Type_prepaid_card,Transaction-Type_student_loan,Transaction-Type_vehicle_loan_or_lease,Transaction-Type_virtual_currency,cd_No,cd_Yes
29880,0.373422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
23505,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
42701,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,0
35679,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,1,0
36005,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,random_state = 0)


In [25]:
from sklearn.multiclass import OneVsRestClassifier

In [37]:
gb = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train_internal)
pred_internal = gb.predict(X_train)
import sklearn
sklearn.metrics.f1_score(y_train_internal, pred_internal,average=None)

array([0.        , 0.88581625, 0.09841677, 0.        , 0.        ])

In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

# Train the model on training data
rm = rf.fit(X_train, y_train_internal)
pred_internal = rm.predict(X_test)



In [ ]:
# import sklearn
sklearn.metrics.f1_score(y_test_internal, pred_internal,average='weighted')

In [ ]:
from sklearn.svm import SVC
sv = SVC(kernel =  'rbf')
sv.fit(X_train, y_train_internal)
pred_internal = sv.predict(X_test)

In [ ]:
import sklearn
sklearn.metrics.f1_score(y_test_internal, pred_internal,average = 'weighted')

In [ ]:
clf = MultinomialNB().fit(x_train, y_train)

In [ ]:
id_ = list(test_data['Complaint-ID'])


In [ ]:
test_data.head()
test_data.drop(['Complaint-Status','Transaction-Type','Complaint-ID'],inplace=True,axis=1)

In [ ]:
y_pred = clf.predict(test_data)

In [ ]:
import sklearn
sklearn.metrics.f1_score(y_test, y_pred,average='weighted')

In [ ]:
result = pd.DataFrame(id_,columns=['Complaint-ID'])
result['Complaint-Status'] = y_pred

In [ ]:
result.head()

In [ ]:
result.to_csv('/home/prashant/Downloads/c3cc8568-0-dataset/baseline.csv',index=None)

In [1]:
result

NameError: name 'result' is not defined